# 1. 데이터 전처리

## 필요 라이브러리를 로드합니다.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from datetime import datetime
import datetime
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from matplotlib import font_manager, rc
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', None)

from tqdm import tqdm_notebook
from tqdm import tqdm
import os
import sys
import urllib.request
import json
from tabulate import tabulate
from konlpy.tag import Kkma

## 시각화를 위한 기초 설정을 합니다.

In [2]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
mpl.rcParams['axes.unicode_minus'] = False
import platform
plt.rc('font', family='NanumBarunGothic') 

## 데이터를 불러들어와 전처리를 시작합니다.

In [3]:
train = pd.read_excel('./01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx', skiprows=1)
test = pd.read_excel('./02_평가데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx', skiprows=1)

In [4]:
# train셋의 취급액 이름 공백을 지워줍니다.
train = train.rename(columns = {' 취급액 ':'취급액'})
# 전처리를 한 번에 해주기 위해서 train, test를 합쳐줍니다.
df_all = pd.concat([train, test]).reset_index()

## 방송 편성 관련 변수를 생성합니다.

In [5]:
# 방송노출 시간 및 횟수와 관련된 변수를 생성합니다.
def make_cast_time(train):
    train['취급액'] = train['취급액'].fillna('0') # 취급액이 nan값인 경우 0으로 대체합니다.
    train = train[train.상품군 != '무형'] # 상품군이 무형인 경우 취급액이 0이므로 제외하고 분석을 진행합니다.
    # nan값이 있는 데이터를 따로 저장해둡니다.
    train_yes_nan = train[train['노출(분)'].isnull()]
    # nan값 없는 데이터를 저장해줍니다.
    train_not_nan = train.dropna()

    train_not_nan_index = train_not_nan.index
    train_not_nan_reset_index = train_not_nan.reset_index(drop=True)

    cast_time = []
    cast_count = []
    time = 0.
    count = 1
    # 방송 누적 노출을 cast_time, 방송 누적 횟수를 cast_count에 저장합니다.
    for i in range(len(train_not_nan_reset_index)):
        if i == 0:
            time = train_not_nan_reset_index['노출(분)'][i] 
            count = 1
            cast_time.append(time)
            cast_count.append(count)
        elif train_not_nan_reset_index['상품코드'][i] == train_not_nan_reset_index['상품코드'][i-1]:
            time += train_not_nan_reset_index['노출(분)'][i]
            count += 1 
            cast_time.append(time) 
            cast_count.append(count) 
        else: 
            time = train_not_nan_reset_index['노출(분)'][i]
            count = 1
            cast_time.append(time) 
            cast_count.append(count)

    train_not_nan_reset_index['cast_time'] = cast_time
    train_not_nan_reset_index['cast_count'] = cast_count
    
    # 해당 방송의 전체 편성 시간 및 편성 횟수를 저장합니다.
    cast_time_sum = []
    cast_count_sum = []

    for i in range(len(train_not_nan_reset_index)):
        if i == max(train_not_nan_reset_index.index):
            cast_time_sum.append(train_not_nan_reset_index['cast_time'][i])
            cast_count_sum.append(train_not_nan_reset_index['cast_count'][i])
        elif train_not_nan_reset_index['상품코드'][i] == train_not_nan_reset_index['상품코드'][i+1]:
            cast_time_sum.append(np.nan)
            cast_count_sum.append(np.nan)
        else: 
            cast_time_sum.append(train_not_nan_reset_index['cast_time'][i])
            cast_count_sum.append(train_not_nan_reset_index['cast_count'][i])
            
    train_not_nan_reset_index['cast_time_sum'] = cast_time_sum
    train_not_nan_reset_index['cast_count_sum'] = cast_count_sum
    train_not_nan_reset_index.fillna(method='bfill', inplace=True)
    

    train_not_nan_reset_index.index = train_not_nan_index
    train_concat = pd.concat([train_not_nan_reset_index, train_yes_nan], axis=0)
    train_concat_sort_index = train_concat.sort_index()
    train = train_concat_sort_index.fillna(method='ffill')
    
    train['cast_time_ratio'] = train.cast_time / train.cast_time_sum # 누적 방송 시간 비율 칼럼을 만들어줍니다.
    train = train.reset_index(drop=True)
    train['cast_count'] = train.cast_count.astype(np.int16)
    
    return train

df_all = make_cast_time(df_all)

In [6]:
# 기본 전처리 함수입니다.
# 취급액과 판매단가의 불필요 요소를 삭제한 후 int 형식으로 데이터 타입을 맞춰줍니다.

def preprocessing_1(train):
    train['취급액'] = train['취급액'].astype(str)
    train['판매단가'] = train['판매단가'].astype(str)
    train['취급액'] = train['취급액'].apply(lambda x: x.replace(',', ''))
    train['판매단가'] = train['판매단가'].apply(lambda x: x.replace(',', ''))
    train['취급액'] = train['취급액'].astype(float) # 소수점 이하 단위를 읽기 위해 임시로 실수형태로 변환합니다.
    train['판매단가'] = train['판매단가'].astype(float) # 소수점 이하 단위를 읽기 위해 임시로 실수형태로 변환합니다.
    train['취급액'] = train['취급액'].astype(np.int32)
    train['판매단가'] = train['판매단가'].astype(np.int32)
    train['판매단가_cat'] = train['판매단가'].apply(lambda x: 0 if x <= 15000 else # 판매단가의 구간을 분할합니다.
                                             (1 if x<=30000 else (2 if x <=45000 else (3 if x <= 60000 else 4))))
    
    # 상품별 최대 판매단가 및 할인정보를 추가합니다.
    tmp = train.groupby('상품명').max()[['판매단가']].reset_index()
    train = pd.merge(train, tmp, on='상품명', how='left')
    train.rename({'판매단가_x':'판매단가','판매단가_y':'할인'}, axis=1, inplace= True)
    train['할인여부'] = train['할인'] - train['판매단가']
    train.drop(['할인'], axis=1, inplace=True)
    
    # 가격이 끝에 9xxxx 으로 끝나는 상품을 컬럼화합니다.
    train['판매단가'] = train['판매단가'].astype(str)
    train.loc[train['판매단가'].str.contains('900') == True, "가격_9x"] = '1'
    train.loc[train['판매단가'].str.contains('900') == False, "가격_9x"] = '0'
    train['판매단가'] = train['판매단가'].astype(int)
    return train

df_all = preprocessing_1(df_all)
df_all.shape

(40088, 17)

In [7]:
# 상품별 최대 판매단가 및 할인정보를 추가합니다.
tmp = df_all.groupby('상품명').max()[['판매단가']].reset_index()
df_all = pd.merge(df_all, tmp, on='상품명', how='left')
df_all.rename({'판매단가_x':'판매단가','판매단가_y':'할인'}, axis=1, inplace= True)
df_all['할인여부'] = df_all['할인'] - df_all['판매단가']
df_all.drop(['할인'], axis=1, inplace=True)

In [8]:
# 상품명을 전처리해 새로운 칼럼을 생성합니다.
def preprocessing_3(train):
    # 1. 일시불/무이자/없음
    train.loc[train['상품명'].str.contains('일시불') == True, "상품명_plan"] = '1'
    train.loc[train['상품명'].str.contains('일\)') == True, "상품명_plan"] = '1'
    train.loc[train['상품명'].str.contains('무이자') == True, "상품명_plan"] = '2'
    train.loc[train['상품명'].str.contains('무\)') == True, "상품명_plan"] = '2'
    train['상품명_plan'] = train['상품명_plan'].fillna('0')

    # 2. 추가구성/단품구성
    train.loc[train['상품명'].str.contains('\+') == True, "상품명_add"] = '1'
    train['상품명_add'] = train['상품명_add'].fillna('0')

    # 3. 기타/삼성/LG
    train.loc[train['상품명'].str.contains('삼성') == True, "상품명_maker"] = '1'
    train.loc[train['상품명'].str.contains('LG') == True, "상품명_maker"] = '2'
    train['상품명_maker'] = train['상품명_maker'].fillna('0')

    # 4. 세트구성/단품구성
    train.loc[train['상품명'].str.contains('세트') == True, "상품명_set"] = '1'
    train['상품명_set'] = train['상품명_set'].fillna('0')

    # 5. 여성/남성/없음
    train.loc[train['상품명'].str.contains('여성') == True, "상품명_sex"] = '1'
    train.loc[train['상품명'].str.contains('브라') == True, "상품명_sex"] = '1'
    train.loc[train['상품명'].str.contains('란쥬') == True, "상품명_sex"] = '1'
    train.loc[train['상품명'].str.contains('블라우스') == True, "상품명_sex"] = '1'
    train.loc[train['상품명'].str.contains('밍크') == True, "상품명_sex"] = '1'
    train.loc[train['상품명'].str.contains('남성') == True, "상품명_sex"] = '2'
    train.loc[train['상품명'].str.contains('드로즈') == True, "상품명_sex"] = '2'
    train.loc[train['상품명'].str.contains('트렁크') == True, "상품명_sex"] = '2'
    train['상품명_sex'] = train['상품명_sex'].fillna('0')
    
    # 6. 아동/성인
    train.loc[train['상품명'].str.contains('주니어') == True, "상품명_kid"] = '1'
    train.loc[train['상품명'].str.contains('여아') == True, "상품명_kid"] = '1'
    train.loc[train['상품명'].str.contains('남아') == True, "상품명_kid"] = '1'
    train.loc[train['상품명'].str.contains('아동') == True, "상품명_kid"] = '1'
    train['상품명_kid'] = train['상품명_kid'].fillna('0')

    # 데이터 타입을 category로 변경합니다.
    train['상품명_plan'] = train['상품명_plan'].astype('category')
    train['상품명_add'] = train['상품명_add'].astype('category')
    train['상품명_maker'] = train['상품명_maker'].astype('category')
    train['상품명_set'] = train['상품명_set'].astype('category')
    train['상품명_sex'] = train['상품명_sex'].astype('category')
    
    return train

df_all = preprocessing_3(df_all)
df_all.shape

(40088, 23)

In [9]:
# 동일 방송에서 여러 상품을 판매하는 경우를 구분하기 위한 칼럼을 생성합니다.

def make_fake_weight(train):
    fake_weight = []
    weight = 1
    for i in range(len(train)):
        if i == 0:
            weight = 1 
            fake_weight.append(weight) 
            
        elif train['마더코드'][i] == train['마더코드'][i-1]: # 자기의 마더코드가 앞행의 것과 같을 때
            if train['판매단가'][i] < train['판매단가'][i-1]: # 자기의 단가가 앞행의 것보다 작으면
                weight +=1 
                fake_weight.append(weight) 
            else: 
                weight = 1 
                fake_weight.append(weight)
        else: 
            weight = 1 
            fake_weight.append(weight)
            
    train['fake_weight'] = fake_weight
    train['fake_weight'] = train.fake_weight.apply(lambda x: -(x*x))
    
    return train

df_all = make_fake_weight(df_all)
df_all.shape

(40088, 24)

In [10]:
# 더 정확한 변수 생성을 위해 기계가 걸러내지 못하는 조건을 필터링합니다.

def make_train_fake(train):
    train_fake = train[train.상품명_plan == '0'] # 할부플랜이 다른 데이터 추리기
    train_fake = train_fake[train_fake.상품명_sex == '0'] # 성별이 다른 데이터 추리기
    train_fake.reset_index(drop=True, inplace=True)

    return train_fake

In [11]:
# 동일 방송에서 여러 상품을 판매하는 경우에 대한 두 번째 변수를 생성합니다.

def make_fake_weight2(train):
    fake_weight = []
    weight = 0
    for i in range(len(train)):
        if i == 0:
            weight = 0 
            fake_weight.append(weight) 
            
        elif train['마더코드'][i] == train['마더코드'][i-1]: # 자기의 마더코드가 앞행의 것과 같을 때
            if train['판매단가'][i] == train['판매단가'][i-1]: # 자기의 단가가 앞행의 것과 같을 때
                if train['방송일시'][i] == train['방송일시'][i-1]:
                    if train['상품코드'][i] != train['상품코드'][i-1]:
                        weight -=1 
                        fake_weight.append(weight) 
                    else:
                        weight = 0
                        fake_weight.append(weight)
                else: 
                    weight = 0 
                    fake_weight.append(weight)
            else:
                weight = 0 
                fake_weight.append(weight)
        else: 
            weight = 0
            fake_weight.append(weight)
            
    train['fake_weight2'] = fake_weight
    
    return train

In [12]:
# 변수를 만들기 위해 생성한 함수를 순서대로 적용합니다.
df_fake = make_train_fake(df_all)
df_fake = make_fake_weight2(df_fake)
df_all.shape

# 기존 데이터에 합쳐줍니다.
df_all = pd.merge(df_all, df_fake[['방송일시','마더코드','상품코드','상품명','판매단가', 'fake_weight2']],
                 on=['방송일시','마더코드','상품코드','상품명','판매단가'], how='left')

# 빈 값은 전부 0으로 채워줍니다.
df_all['fake_weight2'] = df_all['fake_weight2'].fillna(0)

In [13]:
# 마지막 방송 편성 관련 변수를 생성합니다.

def fake_weight3(train):
    # 같은 시간에 방송한 상품에 같은 index를 부여합니다(same).
    train['same'] = 1
    for i in tqdm_notebook(range(1,len(train))):
        if train['cast_count'][i] == train['cast_count'][i-1]:
            train['same'][i] = train['same'][i-1]
        else:
            train['same'][i] = train['same'][i-1] + 1

    # 같은 시간에 여러 품목 방송했는지 알기 위해 cast_count_com 변수를 새로 생성합니다.
    train = pd.merge(train, train.groupby('same').sum().reset_index()[['same','cast_count']], on ='same', how='left')
    train.rename({'cast_count_x':'cast_count', 'cast_count_y':'cast_count_com'}, axis=1, inplace= True)

    # 최종 비교를 위해 com을 한 번 더 생성합니다.
    train['com'] = train['cast_count'] - train['cast_count_com']

    # 같은 방송 여러 품목에 fw라는 순번을 붙여줍니다.
    tmp = train.copy()
    tmp = tmp[tmp['com'] != 0]
    tmp.reset_index(drop=True, inplace=True)
    tmp['fw'] = 1
    for i in tqdm_notebook(range(1, len(tmp))):
        if tmp['same'][i] == tmp['same'][i-1]:
            tmp['fw'][i] = tmp['fw'][i-1]+1
        else:
            pass

    # 다시 데이터에 fw를 붙여주어 fake_weight3으로 이름을 붙입니다.
    train = pd.merge(train, tmp[['방송일시','상품명','fw']], on =['방송일시','상품명'], how = 'left')
    train.rename({'fw':'fake_weight3'},axis=1,inplace=True)
    train.drop(['same','cast_count_com'], axis=1, inplace=True)

    train['fake_weight3']=train['fake_weight3'].fillna(0)
    train['fake_weight3'] = train['fake_weight3'].astype(int)

    # 한 방송에 여러상품 포함하면 1 아니면 0이 되도록 com 변수를 조정합니다.
    train['com'] = train['com'].apply(lambda x: 1 if x==0 else 0)
    train['com'].value_counts()
    
    return train
    
df_all = fake_weight3(df_all)

## 네이버 쇼핑몰 API를 이용하여 제품의 상세 카테고리를 다시 할당합니다.

In [14]:
# 혹여나 남아있을 취급액이 nan 값이거나 상품군이 무형인 경우를 처리합니다.
def treatNA(df):
    df = df[df['상품군']!='무형']
    df['취급액'] = df['취급액'].fillna(0)
    df = df.fillna(method='pad')
    return df

In [15]:
# Unique한 상품명을 추린 이후 상품명에 불필요 단어를 제거해 API를 통해 가져올 상품명 리스트를 저장합니다.
def productnm_cleansing(df):
    # 상품명 중복 제거
    uniq_pname=df['상품명'].unique()
    print('\n<# of Unique pnameuct Name> : \t{}건'.format(len(uniq_pname)))
    np.set_printoptions(threshold=sys.maxsize)

    # 추후 조합을 위해 전처리 이전 제품명과 이후 제품명을 저장합니다.
    p_name = pd.DataFrame(data=uniq_pname, columns=['Before Process'])

    # 제품명에서 불필요한 요소 전처리
    ap=[]
    re_stop = re.compile("""\([가-힣]{1,5}\+[가-힣]{1,5}\)|\(?무이자\)?\s?|\(?일시불\)?\s?|\(?초특가\)?\s?|\(?무\)\s?|\(?유\)\s?|\(?일\)\s?|무료체험|
    |포함|국내[가-힣]+\s|무료설치|\s?신제품\s?|\s?패키지\s?|[0-9]+종|풀코디|set|SET|풀세트|[0-9]+세트|더블팩|싱글팩|[0-9]{1,2}\+[0-9]{1,2}|[0-9]{1,2}인용|[0-9]{1,2}박스|[0-9\.]{1,4}미터|[0-9\.]{1,4}kg|[0-9\.]{1,4}[kKgG]|[0-9]{1,3}[벌롤종단구대P개통병포미봉팩장gL매]|
    |\s[0-9\.]{1,4}[Mm]|\(.{1,10}\)$|^\(.?\)|^[0-9]{2,4}\s|\s+[0-9]{2,4}\s|[0-9]{2,4}년\s|[0-9]{2,4}년형\s|시즌[0-9]|[0-9]{1,3}\%|\(.{1,20}\)$|기본형|고급형|오리지널|[대중소大中小]형|.{1,10}by|,|\s?총[0-9\s]|^[가-힣a-zA-Z]{3,3}의|
    |S\/S|F\/W|f\/w|s\/s|[가-힣]+형\s|[슈퍼]{0,2}싱글|\s[SQK퀸킹]{1,2}\s|[SQK퀸킹]{1,2}$|[가-힣]{0,2}사이즈""")

    remnants = re.compile('[",g\+lL-]\s|ml|_|\sx|[\[\]\］!"#$%&\'()*+,./:;<=>?@\^_`{|}~-]|\s종$|\s[0-9]+\s|\s{2,}')

    for i in p_name['Before Process']:
        tmp = re_stop.sub(' ',i)
        tmp = tmp.strip()
        tmp = remnants.sub(' ',tmp)
        ap.append(tmp.strip())

    # 전처리된 제품명 중복제거
    p_name['After Process'] = ap
    p_name.to_html('tmp.html')

    ap = list(set(ap))

    display(p_name.head(5))
    
    return p_name

In [29]:
# 조건을 설정한 이후 API를 호출합니다.
def naver_shop(keyword):
    client_id = "tdrEWZrN1ZQp9u0yqy3e"
    client_secret = "59Bl3ltKdX"
    keyword = keyword.strip()
    encText = urllib.parse.quote(keyword)
    url = "https://openapi.naver.com/v1/search/shop.json?query=" + encText + "&display=1" # json 결과
    # url = "https://openapi.naver.com/v1/search/shop.xml?query=" + encText # xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        jsonString = response_body.decode('utf-8')
        j = json.loads(jsonString)
        if int(j['total']):
            c1 = j["items"][0]['category1']
            c2 = j["items"][0]["category2"]
            c3 = j["items"][0]["category3"]
            c4 = j["items"][0]["category4"]
            keyword2=''
        else : # 검색결과가 없을경우 모델명(숫자+알파벳 구성)과 사이즈(주로 한자어)를 제거하고 뒤에서부터 2개의 어절만으로 검색 재시도
            p_code = re.compile('[0-9a-zA-Z]+$|[一-龥]+$')
            keyword2 = p_code.sub(' ',keyword)
            keyword2 = keyword2.strip()
            keyword2 = p_code.sub(' ',keyword2)
            keyword2 = ' '.join(keyword2.split()[-2:])
            
            encText = urllib.parse.quote(keyword2)
            url = "https://openapi.naver.com/v1/search/shop.json?query=" + encText + "&display=1"
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            rescode = response.getcode()
            
            if(rescode==200):
                response_body = response.read()
                jsonString = response_body.decode('utf-8')
                j = json.loads(jsonString)
                if int(j['total']):
                    c1 = j["items"][0]['category1']
                    c2 = j["items"][0]["category2"]
                    c3 = j["items"][0]["category3"]
                    c4 = j["items"][0]["category4"]
                else :
                    print(keyword,'->',keyword2)
                    print('>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<')
                    c1, c2, c3, c4 = '', '', '', ''
            else :
                print("Error Code:" + rescode)
    else:
        print("Error Code:" + rescode)
    return [keyword,c1,c2,c3,c4,keyword2]

In [30]:
# 형태소 분석기를 이용해 주요 단어만을 추출한 이후 결과를 비교합니다.
def jaccard_sim(doc1, doc2):
    kkma = Kkma()
    doc1 = kkma.nouns(doc1)
    doc2 = kkma.nouns(doc2)
    u_doc1 = set(doc1)
    u_doc2 = set(doc2)
    return len(u_doc1 & u_doc2) / len(u_doc1 | u_doc2)

In [31]:
# 생성한 함수를 기반으로 실제 API 사용을 위한 함수를 정의합니다.
def cat_finder(df):
    print("전처리중...")
    treated = treatNA(df)
    df_clean = productnm_cleansing(treated)
    pnames = set(df_clean['After Process'].tolist())
    cat_df = pd.DataFrame(columns = ['pname','cat1','cat2','cat3','cat4','pname2'])
    
    print("1차 분류 시작...")
    for item in tqdm_notebook(pnames):
        ns = naver_shop(item)
        cat_df = cat_df.append(pd.DataFrame([ns],columns=['pname','cat1','cat2','cat3','cat4','pname2']), ignore_index=True)
    unknown_ratio = len(cat_df[cat_df['cat1']==''])/len(cat_df)*100
    print("1차 분류 완료 \n1차 분류에서 알아내지 못한 카테고리 비율 : ", round(unknown_ratio,3),"%")
    
    print("2차 분류 시작...")
    unknown = cat_df[cat_df['cat1']=='']
    known = cat_df[cat_df['cat1']!='']
    
    for i in tqdm_notebook(range(len(unknown))):
        best_sim = 0
        print(unknown.iloc[i].pname)
        for j in tqdm_notebook(range(len(known))):
            sim = jaccard_sim(unknown.iloc[i].pname,known.iloc[j].pname)
            if sim > best_sim :
                best_sim = sim
                best_ind = j
                print(best_sim)
        unknown.iloc[i].cat1 = known.iloc[best_ind].cat1
        unknown.iloc[i].cat2 = known.iloc[best_ind].cat2
        unknown.iloc[i].cat3 = known.iloc[best_ind].cat3
        unknown.iloc[i].cat4 = known.iloc[best_ind].cat4
        print(known.iloc[best_ind].cat1, known.iloc[best_ind].cat2,known.iloc[best_ind].cat3,known.iloc[best_ind].cat4)
    print("2차 분류 완료")
    print("원래 데이터와 병합중...")
    # 여기물어보기
    tmp = pd.merge(df_clean,pd.concat([known,unknown]),left_on = 'After Process', right_on = 'pname').drop(['pname','pname2','After Process'],axis=1)
    re_df = pd.merge(df,tmp,left_on='상품명',right_on='Before Process').drop(['Before Process'],axis=1)
    display(re_df.head())
    print("병합 완료!")
    return re_df


In [32]:
category = cat_finder(df_all)

전처리중...

<# of Unique pnameuct Name> : 	1998건


,Before Process,After Process
0,테이트 남성 셀린니트3종,테이트 남성 셀린니트
1,테이트 여성 셀린니트3종,테이트 여성 셀린니트
2,오모떼 레이스 파운데이션 브라,오모떼 레이스 파운데이션 브라
3,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,PAT 남성 소프트 기모 릴렉스팬츠
4,보코 리버시블 무스탕,보코 리버시블 무스탕


1차 분류 시작...


기간 제주바다자연산돔39마리 -> 기간 제주바다자연산돔39마리
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
USPA화이트라벨 여성트랙수트 -> USPA화이트라벨 여성트랙수트
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
우리바다 손질왕꼬막 -> 우리바다 손질왕꼬막
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
The귀한 소한마리세트 -> The귀한 소한마리세트
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
디키즈 썸머상하세트 -> 디키즈 썸머상하세트
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
마르엘라로사티 에코무스탕 -> 마르엘라로사티 에코무스탕
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
아가타 컬러풀에디션 -> 아가타 컬러풀에디션
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
남해바다 손질왕꼬막살세트 -> 남해바다 손질왕꼬막살세트
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
탕요일 -> 탕요일
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
황제진액 철갑상어 -> 황제진액 철갑상어
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
메이듀 여성 린넨 블렌디드 슬립온 -> 블렌디드 슬립온
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
파격가 니봇 무선 진공 물걸레 로봇청소기  뉴헤어드라이어기 -> 로봇청소기 뉴헤어드라이어기
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<

기간 제주바다자연산돔39마리


...

0.06666666666666667
0.07142857142857142
0.5454545454545454

패션의류 남성의류 트레이닝복 
한삼인순홍삼진



패션의류 남성의류 트레이닝복 
에트로 프로푸미 웨이스트백


0.07692307692307693
0.08333333333333333
0.09090909090909091
0.2
0.5555555555555556

패션잡화 여성가방 크로스백 

2차 분류 완료
원래 데이터와 병합중...


,cast_count,cast_count_sum,cast_time,cast_time_sum,index,노출(분),마더코드,방송일시,상품군,상품명,상품코드,취급액,판매단가,cast_time_ratio,판매단가_cat,할인여부,가격_9x,상품명_plan,상품명_add,상품명_maker,상품명_set,상품명_sex,상품명_kid,fake_weight,fake_weight2,com,fake_weight3,cat1,cat2,cat3,cat4
0,1,3.0,20.0,60.0,0,20.0,100346,2019-01-01 06:00:00,의류,테이트 남성 셀린니트3종,201072,2099000,39900,0.333333,2,0,1,0,0,0,0,2,0,-1,0.0,0,1,패션의류,남성의류,니트/스웨터,
1,2,3.0,40.0,60.0,2,20.0,100346,2019-01-01 06:20:00,의류,테이트 남성 셀린니트3종,201072,3262000,39900,0.666667,2,0,1,0,0,0,0,2,0,-1,0.0,0,1,패션의류,남성의류,니트/스웨터,
2,3,3.0,60.0,60.0,4,20.0,100346,2019-01-01 06:40:00,의류,테이트 남성 셀린니트3종,201072,6672000,39900,1.000000,2,0,1,0,0,0,0,2,0,-1,0.0,0,1,패션의류,남성의류,니트/스웨터,
3,1,2.0,20.0,40.0,164,20.0,100346,2019-01-03 00:00:00,의류,테이트 남성 셀린니트3종,201072,7329000,39900,0.500000,2,0,1,0,0,0,0,2,0,-1,0.0,0,1,패션의류,남성의류,니트/스웨터,
4,2,2.0,40.0,40.0,166,20.0,100346,2019-01-03 00:20:00,의류,테이트 남성 셀린니트3종,201072,10481000,39900,1.000000,2,0,1,0,0,0,0,2,0,-1,0.0,0,1,패션의류,남성의류,니트/스웨터,


병합 완료!


In [33]:
# 결측치를 먼저 채워 줍니다.
category['cat3'] = category.apply(lambda x: x['cat2'] if x['cat3'] != x['cat3'] else x['cat3'], axis=1)

# 원래 데이터와 병합을 위해 key값을 생성해줍니다.
def preprocessing_to_merge(df):
    # merge를 위해 datetime으로 바꿔줍니다..
    df['방송일시'] = pd.to_datetime(df['방송일시'], errors='coerce')
    df['방송일시'] = df['방송일시'].astype(str)    
    # key 칼럼을 만들어줍니다. 방송일시만으로는 안되니, 방송일시+상품명을 해줍니다.
    df['key'] = df['방송일시'] + df['상품명']
    return df

# 두 데이터를 병합합니다.
def preprocessing_merging(df1, df2):
    # df1이 왼쪽, df2가 오른쪽입니다.
    df3 = pd.merge(df1, df2[['key','cat1','cat2','cat3']], how='left', on=['key'])
    df3 = df3.drop('key', axis=1) # key 칼럼은 드랍.
    
    return df3

In [34]:
# 생성한 함수를 이용해 두 데이터의 병합을 진행합니다.
df_all = preprocessing_to_merge(df_all)
category1 = preprocessing_to_merge(category)
df_all = preprocessing_merging(df_all, category1)
df_all.shape

(40088, 30)

## 방송일시와 관련된 변수를 생성합니다.

In [35]:
# 방송일시를 분할하여 변수를 새롭게 생성합니다.
def preprocessing_4(train):
    train['방송일시'] = pd.to_datetime(train['방송일시'], errors='coerce')

    # 월일이 다음날로 넘어가는 것을 막기 위해서 3시간씩 앞으로 땡겨줍니다. 마지막에 다시 더해줘야 합니다.
    delta = datetime.timedelta(hours=3)
    train['방송일시'] = train.방송일시.apply(lambda x: x - delta)

    # dayofweek는 날짜에서 요일(월~일)을 가져오는 기능입니다.
    # 값은 0(월), 1(화), 2(수), 3(목), 4(금), 5(토), 6(일) 을 나타냅니다.
    train["방송일시_dow"] = train["방송일시"].dt.dayofweek

    # 요일 외에 다른 정보를 가져오기 위해서 다시 string으로 바꿔줍니다.
    train['방송일시'] = train.방송일시.astype(str)

    # MMDDhhmm 정보를 가져옵니다.
    train['방송일시_MM'] = train['방송일시'].apply(lambda x: x[5:7])
    train['방송일시_DD'] = train['방송일시'].apply(lambda x: x[8:10])
    train['방송일시_hh'] = train['방송일시'].apply(lambda x: x[11:13])
    train['방송일시_mm'] = train['방송일시'].apply(lambda x: x[14:16])

    # schedule 관련 칼럼을 더 만들어보겠습니다. # MMDD # DDHH # HHMM # weekday/weekends
    train['방송일시_MMDD'] = train['방송일시_MM'] + train['방송일시_DD']
    train['방송일시_DDhh'] = train['방송일시_DD'] + train['방송일시_hh']
    train['방송일시_hhmm'] = train['방송일시_hh'] + train['방송일시_mm']
    train['방송일시_MMDDhh'] = train['방송일시_MM'] + train['방송일시_DD'] + train['방송일시_hh']

    # mmmm_1 은 1일 사이클로 분단위로 환산한 것
    # mmmm_2 는 1달 사이클로 분단위로 환산한 것
    # mmmm_3 는 1년 사이클로 분단위로 환산한 것
    train['방송일시_mmmm_1'] = train['방송일시_hh'].astype(int) * train['방송일시_mm'].astype(int) * 60
    train['방송일시_mmmm_2'] = train['방송일시_DD'].astype(int) * train['방송일시_hh'].astype(int) * train['방송일시_mm'].astype(int) * 60
    train['방송일시_mmmm_3'] = train['방송일시_MM'].astype(int) * train['방송일시_DD'].astype(int) * train['방송일시_hh'].astype(int) * train['방송일시_mm'].astype(int) * 60

    # weekday = 1 / weekends = 0
    train.loc[train.방송일시_dow == 5, '방송일시_dow2'] = '0'
    train.loc[train.방송일시_dow == 6, '방송일시_dow2'] = '0'
    train['방송일시_dow2'] = train.방송일시_dow2.fillna('1')
    
    # 데이터 타입을 변환합니다.
    train['방송일시_dow'] = train.방송일시_dow.astype(np.int16)
    train['방송일시_MM'] = train.방송일시_MM.astype('category')
    train['방송일시_DD'] = train.방송일시_DD.astype('category')
    train['방송일시_hh'] = train.방송일시_hh.astype('category')
    train['방송일시_mm'] = train.방송일시_mm.astype('category')
    train['방송일시_MMDD'] = train.방송일시_MMDD.astype('category')
    train['방송일시_DDhh'] = train.방송일시_DDhh.astype('category')
    train['방송일시_hhmm'] = train.방송일시_hhmm.astype('category')
    train['방송일시_dow2'] = train.방송일시_dow2.astype('category')
    
    return train

df_all = preprocessing_4(df_all)
df_all.shape

(40088, 43)

In [36]:
# 방송시간을 기준으로 카테고리 변수를 새로 생성합니다.
list_0 = ['03', '04', '05', '06', '07','08'] # 낮12시 이전
list_1 = ['09', '10', '11', '12', '13','14', '15', '16', '17'] # 낮 12시부터
list_2 = ['18', '19', '20', '21', '22', '23'] # 밤 9시부터
list_4 = ['03', '21', '22', '23'] # 심야시간대, 03, 21, 22, 23
list_5 = ['04', '05', '06', '07', '08', '09'] # 낮 12시까지 오전
list_6 = ['10', '11', '12', '13', '14', '15'] # 저녁 18시 이전까지 오후, 나머진 저녁

df_all['time_cat'] = df_all['방송일시_hh'].apply(lambda x:0 if x in list_0 else(1 if x in list_1 else (2 if x in list_2 else 3)))
df_all['time_cat2'] = df_all['방송일시_hh'].apply(lambda x:0 if x in list_4 else(1 if x in list_5 else (2 if x in list_6 else 3)))

# 시간대별 평균 취급액을 추가합니다.
tmp = df_all.groupby('방송일시_hhmm').mean()[['취급액']].reset_index()
df_all = pd.merge(df_all, tmp, on='방송일시_hhmm', how='left')
df_all.rename({'취급액_y':'mean_amt_by_hhmm','취급액_x':'취급액'}, axis=1, inplace=True)
df_all.shape

(40088, 46)

## 상품명과 관련한 변수를 생성합니다.

In [37]:
#상품명에서 불필요 단어를 제외한 새로운 상품명, 상품의 브랜드 정보를 추출합니다.

def pre_nlp(train):
    # 제품명에서 불필요한 요소 전처리
    new_product_name = []
    re_stop = re.compile("""\([가-힣]{1,5}\+[가-힣]{1,5}\)|\(?무이자\)?\s?|\(?일시불\)?\s?|\(?초특가\)?\s?|\(?무\)\s?|\(?유\)\s?|\(?일\)\s?|무료체험|
    |포함|국내[가-힣]+\s|무료설치|\s?신제품\s?|\s?패키지\s?|[0-9]+종|풀코디|set|SET|풀세트|[0-9]+세트|더블팩|싱글팩|[0-9]{1,2}\+[0-9]{1,2}|[0-9]{1,2}인용|[0-9]{1,2}박스|[0-9\.]{1,4}미터|[0-9\.]{1,4}kg|[0-9\.]{1,4}[kKgG]|[0-9]{1,3}[벌롤종단구대P개통병포미봉팩장gL매]|
    |\s[0-9\.]{1,4}[Mm]|\(.{1,10}\)$|^\(.?\)|^[0-9]{2,4}\s|\s+[0-9]{2,4}\s|[0-9]{2,4}년\s|[0-9]{2,4}년형\s|시즌[0-9]|[0-9]{1,3}\%|\(.{1,20}\)$|기본형|고급형|오리지널|[대중소大中小]형|.{1,10}by|,|\s?총[0-9\s]|^[가-힣a-zA-Z]{3,3}의|
    |S\/S|F\/W|f\/w|s\/s|[가-힣]+형\s|[슈퍼]{0,2}싱글|\s[SQK퀸킹]{1,2}\s|[SQK퀸킹]{1,2}$|[가-힣]{0,2}사이즈""")
    remnants = re.compile('[",g\+lL-]\s|ml|_|\sx|[\[\]\］!"#$%&\'()*+,./:;<=>?@\^_`{|}~-]|\s종$|\s[0-9]+\s|\s{2,}')

    for i in train.상품명:
        tmp = re_stop.sub(' ',i)
        tmp = tmp.strip()
        tmp = remnants.sub(' ',tmp)
        new_product_name.append(tmp.strip())
    
    train['new_상품명'] = new_product_name

    # 상품명에서 브랜드 추출
    train['상품명_brand'] = train['new_상품명'].apply(lambda x: x[:2])
    
    return train

df_all = pre_nlp(df_all)
df_all.shape

(40088, 48)

## 네이버 쇼핑의 데이터를 크롤링해 새로운 정보를 가져옵니다.

In [38]:
# 원 데이터의 상품명을 네이버 쇼핑에 검색하는 로직을 생성합니다.
name = df_all['상품명'].unique().tolist()
url1 = 'https://search.shopping.naver.com/search/all?query='
url2 = '&cat_id=&frm=NVSHATC'

# 네이버 쇼핑에서 같은 상품의 가격 정보와 리뷰 개수를 가져옵니다.
price = []
review = []
for i in tqdm_notebook(range(len(name))):
    r=requests.get(url1+name[i]+url2)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    review1 = soup.find_all('em', 'basicList_num__1yXM9', limit=1)
    price1 = soup.find_all('span', 'price_num__2WUXn', limit=1)
    if len(price1) == 1:
        price.append(price1[0].text)
    else:
        price.append(0)
    if len(review1) == 1:
        review.append(review1[0].text)
    else:
        review.append(0)

In [39]:
# 가져온 정보를 새로운 Dataframe으로 변환합니다.
naver = pd.DataFrame()
naver['상품명'] = name
naver['review_counts'] = review
naver['internet_price'] = price

# 중복 값은 제거합니다.
naver = naver.drop_duplicates(subset=['상품명'])

In [40]:
# 크롤링 결과를 원 데이터에 합친 이후 전처리합니다.
def outside_feature_naver(train, naver):
    train = pd.merge(train, naver, on='상품명', how='left')
    
    # 불필요 요소를 삭제한 이후 데이터 타입을 int로 변경합니다.
    train['review_counts'] = train['review_counts'].astype(str)
    train['internet_price'] = train['internet_price'].astype(str)
    train['review_counts'] = train['review_counts'].apply(lambda x: x.replace(',',''))
    train['internet_price'] = train['internet_price'].apply(lambda x: x.replace(',',''))
    train['internet_price'] = train['internet_price'].apply(lambda x: x.replace('원',''))
    train['review_counts'] = train['review_counts'].astype(int)
    train['internet_price'] = train['internet_price'].astype(int)

    # 오차를 컬럼으로 만들어줍니다.
    train['price_minus'] = train['판매단가'] - train['internet_price']

    # 네이버에 서칭이 되는지 여부를 구분합니다.
    train['search_naver'] = train['internet_price'] + train['review_counts']
    train['search_naver'] = train['search_naver'].apply(lambda x: 0 if x == 0 else 1)
    
    return train

df_all = outside_feature_naver(df_all, naver)
df_all.shape

(40088, 52)

## 기상 관련 정보를 이용해 새로운 변수를 생성합니다. (데이터는 기상청 제공 파일입니다.)

In [41]:
# 기상청 통계 파일 읽기
weather_train = pd.read_csv('weather_train.csv', encoding='latin1')
weather_test = pd.read_csv('weather_test.csv', encoding='latin1')
weather = pd.concat([weather_train, weather_test], ignore_index=True)
columns = ['location_id','location','date','temperature']
weather.columns = columns

In [42]:
# 날씨 정보를 원 데이터와 합치기위해 시간을 기준으로 데이터를 분할합니다.
def outside_feature_weather(weather):
    
    weather['date'] = pd.to_datetime(weather['date'], errors='coerce')

    # 월일이 다음날로 넘어가는 것을 막기 위해서 3시간씩 앞으로 땡겨줍니다. 마지막에 다시 더해줘야 합니다.
    delta = datetime.timedelta(hours=3)
    weather['date'] = weather.date.apply(lambda x: x - delta)

    # 요일 외에 다른 정보를 가져오기 위해서 다시 string으로 바꿔줍니다.
    weather['date'] = weather.date.astype(str)
    
    # key column 만들어주어 데이터를 병합할 수 있도록 합니다.
    weather['weather_key'] = (
        weather['date'].apply(lambda x: x[2:4]) + 
        weather['date'].apply(lambda x: x[5:7]) + 
        weather['date'].apply(lambda x: x[8:10]) + 
        weather['date'].apply(lambda x: x[11:13])
    )

    # 필요없는 칼럼은 미리 버려줍니다.
    weather.drop(['location_id', 'location', 'date'], axis=1, inplace=True)
    
    return weather

In [43]:
# 생성한 함수를 이용해 날씨 정보를 합쳐줍니다.
weather = outside_feature_weather(weather)
# key column 만들어주기
df_all['weather_key'] = (
    df_all['방송일시'].apply(lambda x: x[2:4]) + 
    df_all['방송일시'].apply(lambda x: x[5:7]) + 
    df_all['방송일시'].apply(lambda x: x[8:10]) + 
    df_all['방송일시'].apply(lambda x: x[11:13])
)

df_all = pd.merge(df_all, weather, on='weather_key', how='left')

# 관측이 안 된 날이 있어 그 전 시점의 날씨로 대체합니다.
df_all['temperature'] = df_all['temperature'].fillna(method='ffill')
# key column을 지워줍니다.
df_all.drop(['weather_key'], axis=1, inplace=True)
df_all.shape

(40088, 53)

## 코스피 지수를 이용한 외부 데이터를 생성합니다. (데이터는 investing.com 제공 파일입니다.)

In [44]:
# train, test 시기의 KOSPI 데이터를 받은 후 이를 합쳐줍니다.
kospi_train = pd.read_csv('KOSPI_train.csv')
kospi_test = pd.read_csv('KOSPI_test.csv')
kospi = pd.concat([kospi_train, kospi_test], ignore_index=True)

In [45]:
# 코스피 지수를 원 데이터와 병합하기 위해 데이터를 처리합니다.
df_all['kospi_key'] = (
    df_all['방송일시'].apply(lambda x: x[2:4]) + 
    df_all['방송일시'].apply(lambda x: x[5:7]) + 
    df_all['방송일시'].apply(lambda x: x[8:10]))
kospi['Date'] = pd.to_datetime(kospi['Date'], errors='coerce')
kospi['Date'] = kospi['Date'].astype(str)
kospi['kospi_key'] = kospi['Date'].apply(lambda x: x[2:4]) + kospi['Date'].apply(lambda x: x[5:7]) + kospi['Date'].apply(lambda x: x[8:10])

def kospi_feature(kospi,df_all):
    kospi = kospi[['Date','Change %','kospi_key']] # 날짜와 전날 대비 코스피 지수의 변동 정보를 가져옵니다.
    kospi['Change %'] = kospi['Change %'].apply(lambda x: x[:-1]) # 변동에서 '%' 문자를 지워줍니다.
    kospi['Change %'] = kospi['Change %'].astype(float) # float 형식으로 데이터 타입을 변경합니다.
    df_all = pd.merge(df_all, kospi, on ='kospi_key',how='left')
    df_all['Change %'] = df_all['Change %'].fillna(0) # 날짜가 없는 경우 변동이 없는 경우이므로 0을 넣습니다.
    
    return df_all

In [46]:
df_all = kospi_feature(kospi,df_all)
df_all.shape

(40088, 56)

## 네이버 datalab을 이용해 NS홈쇼핑 검색 관련 변수를 생성합니다. (데이터는 datalab에서 제공한 파일입니다.)

In [47]:
# train, test 시기의 검색 데이터를 받은 후 이를 합쳐줍니다.
search_train = pd.read_excel('datalab_train.xlsx',skiprows=6)
search_test = pd.read_excel('datalab_test.xlsx',skiprows=6)
search = pd.concat([search_train, search_test], ignore_index=True)

In [48]:
df_all['날짜'] = df_all['방송일시'].apply(lambda x: x[2:4]) + df_all['방송일시'].apply(lambda x: x[5:7]) + df_all['방송일시'].apply(lambda x: x[8:10])

In [49]:
# 검색 데이터를 원 데이터와 합치기 위해 함수를 생성합니다.
def search_data(search, df_all):
    search = search[['날짜','NS홈쇼핑']] # NS 홈쇼핑의 상대적 검생량만을 가져옵니다.
    search['날짜'] = search['날짜'].apply(lambda x: x[2:])
    search['날짜'] = search['날짜'].apply(lambda x: x.replace('-',''))
    df_all = pd.merge(df_all, search, on = '날짜', how = 'left') # 원 데이터와 병합해줍니다.
    df_all.rename({'NS홈쇼핑':'search_compare'},axis=1, inplace= True)
    return df_all

df_all = search_data(search, df_all)
df_all.shape

(40088, 58)

## 카테고리 데이터 인코딩

In [50]:
# 데이터에 category 형식의 변수가 많이 쌓였습니다.
# one-hot encoding을 이용할 경우 변수가 너무 많아질 수 있으므로 label encoding을 준비합니다.
def label_encoding(train_, columns, number):
    length = range(number)
    for c, i in zip(columns, length):
        globals()['encoder_{}:'.format(i)] = LabelEncoder()
        globals()['encoder_{}:'.format(i)].fit(train_[c].values)
        train_['encoding_{}:'.format(c)] = globals()['encoder_{}:'.format(i)].transform(train_[c])

# 인코딩할 컬럼을 리스트로 넣어줍니다.
columns = ['상품명', '상품군', 'cat1', 'cat2', 'cat3', '상품명_brand', 'new_상품명']
number = len(columns)

# 생성한 함수로 인코딩을 처리합니다.
label_encoding(df_all, columns, number)

## 데이터 재분할

In [57]:
# 생성한 데이터를 다시 train test 데이터로 분할합니다.
df_all.drop(['index'], axis=1, inplace=True)
print(df_all.shape)
train = df_all.iloc[:37372]
test = df_all.iloc[37372:]
print(train.shape)
# 예측 값 추정이 불가능한 취급액이 0원인 데이터를 제거해줍니다.
train = train[train.취급액 != 0]

# train 셋의 아웃라이어를 제거합니다.1억 이상의 취급액을 가진 데이터가 약 1% 정도 있습니다.
train = train[train.취급액 < 100000000]
print(train.shape)
print(test.shape)

(40088, 64)
(37372, 64)
(35196, 64)
(2716, 64)


# 2. 모델 학습

In [58]:
# 학습에 이용할 변수를 정의합니다.
# 변수 정의의 자세한 내용은 설명서 및 보조 설명서에 포함하였습니다.
features = ['노출(분)','마더코드', '상품코드', '판매단가', 'cast_time', 'cast_count', 'cast_time_sum', 'cast_count_sum',
             'cast_time_ratio', '상품명_plan', '상품명_add', '상품명_maker', '상품명_set', '상품명_sex', '상품명_kid',
              'fake_weight', 'fake_weight2', '방송일시_hh', '방송일시_MMDD', '방송일시_hhmm', '방송일시_MMDDhh', '방송일시_dow2',
              'review_counts', 'internet_price', 'price_minus', 'search_naver', '가격_9x', '할인여부', 'mean_amt_by_hhmm',
              'time_cat2', '판매단가_cat', 'encoding_상품명:', 'encoding_상품군:', 'encoding_상품명_brand:', 'encoding_new_상품명:',
              'encoding_cat1:', 'encoding_cat2:', 'encoding_cat3:', 'com', 'fake_weight3']

# 종속변수의 이상치를 제거합니다.
train = train[train['취급액'] < 100000000]

# 독립변수와 종속변수를 할당해 학습 준비를 합니다.
X_train = train[features]
y_train = train['취급액'].values
X_test = test[features]

# 로그 변환을 적용합니다.
y_train_log = np.log(y_train)

In [59]:
# XGBoost 모델을 이용해 학습을 진행합니다.
# 최적 하이퍼 파라미터 관련 설명은 설명서에 포함했습니다.
from xgboost import XGBRegressor
X_train = np.array(X_train)
X_test = np.array(X_test)
xgb = XGBRegressor(learning_rate=0.247,
                  max_depth=7,
                  n_estimators=300,
                    colsample_bytree=0.31,
                  random_state=529)

xgb.fit(X_train, y_train_log)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.31, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.247, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=300, n_jobs=0, num_parallel_tree=1, random_state=529,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [72]:
# 테스트 데이터에 예측 결과를 도출한 후 다시 지수변환을 시행합니다.
pred_log = xgb.predict(X_test)
pred_log_T = np.exp(1)**pred_log

In [73]:
# 예측된 수치를 test 데이터에 적용합니다.
test['취급액'] = pred_log_T

In [74]:
# 최종 Submission 파일을 저장합니다.
submission = pd.read_excel('./02_평가데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx', skiprows=1)

# 다시 데이터의 시간을 원상복구 시킵니다.
test['방송일시'] = pd.to_datetime(test['방송일시'], errors='coerce')
delta = datetime.timedelta(hours=3)
test['방송일시'] = test['방송일시'].apply(lambda x: x + delta)

In [75]:
# 두 데이터를 병합시킵니다.
print(submission.shape)
final = pd.merge(submission, test[['방송일시','상품명','취급액']], on =['방송일시','상품명'], how= 'left')
print(final.shape)

(2891, 8)
(2891, 9)


In [76]:
# 데이터 변수를 정리합니다.
final.rename({'취급액_y':'취급액'}, axis=1, inplace=True)
del final['취급액_x']

In [77]:
# 최종적으로 submission file을 확인합니다.
print(final[final['취급액']>1].shape)
final['취급액'] = final['취급액'].fillna(0)
final.isnull().sum()

(2716, 8)


방송일시        0
노출(분)    1111
마더코드        0
상품코드        0
상품명         0
상품군         0
판매단가        0
취급액         0
dtype: int64

In [80]:
final.to_csv('submission.csv',header=True, index = False, encoding = 'cp949')